In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# handwriting number recognition

mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

n_classes = 10
num_feature = 28*28
batch_size = 100 # how many samples you load at a time for training

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [11]:
layer_weight = []

def build_layer(prev_layer, num_node):
    prev_num_node = prev_layer.shape[1].value
    xavier_stddev = tf.sqrt(3.0/(prev_num_node + num_node))
    W = tf.Variable(tf.random_normal(shape=[prev_num_node, num_node], stddev=xavier_stddev))
    b = tf.Variable(tf.zeros([num_node]))
    layer_weight.append(tf.reshape(W,[-1]))
    return tf.add(tf.matmul(prev_layer, W), b)

In [12]:
x = tf.placeholder("float", shape=[None, num_feature])
y_true = tf.placeholder("float", shape=[None, n_classes])

In [13]:
layer1 = build_layer(x, 500)
layer1 = tf.nn.relu(layer1)

layer2 = build_layer(layer1, 500)
layer2 = tf.nn.relu(layer2)

layer3 = build_layer(layer2, 500)
layer3 = tf.nn.relu(layer3)

In [14]:
y = build_layer(layer3, n_classes) #output layer

In [18]:
all_weight = tf.concat(layer_weight, 0)

In [23]:
weight_decay = 0.0003
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true, logits=y)) + 0.5*weight_decay*tf.reduce_sum(tf.square(all_weight))

In [24]:
optimizer = tf.train.AdamOptimizer().minimize(loss)
hm_epochs = 30

In [ ]:
with tf.Session() as s:
    tf.global_variables_initializer().run()

    for epoch in range(hm_epochs):
        epoch_loss = 0
        for _ in range(int(mnist.train.num_examples/batch_size)):
            epoch_x, epoch_y = mnist.train.next_batch(batch_size) 
            _, c = s.run([optimizer, loss], feed_dict={x: epoch_x, y_true: epoch_y})
            epoch_loss += c

        print 'Epoch: ', epoch, ' completed out of ', hm_epochs, ' loss: ', epoch_loss

    correct = tf.equal(tf.argmax(y, 1), tf.argmax(y_true, 1))

    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
    print 'Accuracy: ', accuracy.eval({x: mnist.test.images, y_true: mnist.test.labels})

Epoch:  0  completed out of  30  loss:  256.20293140411377


In [15]:
len(epoch_x[0])

784

In [ ]:
# height x width
x = tf.placeholder('float', [None, 784]) # image is flattened 28 x 28
y = tf.placeholder('float')

def neural_network_model(data):
    hidden_1_layer = {
            'weights': tf.Variable(tf.random_normal([784, n_nodes_hl1])),
            'biases': tf.Variable(tf.random_normal([n_nodes_hl1]))
    }

    hidden_2_layer = {
            'weights': tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
            'biases': tf.Variable(tf.random_normal([n_nodes_hl2]))
    }

    hidden_3_layer = {
            'weights': tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
            'biases': tf.Variable(tf.random_normal([n_nodes_hl3]))
    }

    output_layer = {
            'weights': tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
            'biases': tf.Variable(tf.random_normal([n_classes]))
    }

    l1 = tf.add(tf.matmul(data, hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1) # rectified linear

    l2 = tf.add(tf.matmul(l1, hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2, hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)

    output_layer = tf.add(tf.matmul(l3, output_layer['weights']), output_layer['biases'])

    return output_layer


def train_neural_network(x, y):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))

    # learning_rate = 0.001 default
    optimizer = tf.train.AdamOptimizer().minimize(cost)

    # cycles of feed forward + back prop
    hm_epochs = 30

    with tf.Session() as s:
        s.run(tf.initialize_all_variables())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)

                _, c = s.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

            print 'Epoch: ', epoch, ' completed out of ', hm_epochs, ' loss: ', epoch_loss

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print 'Accuracy: ', accuracy.eval({x: mnist.test.images, y: mnist.test.labels})

train_neural_network(x, y)